Script to finetune BART model
This script generates a fine tuned BART model from the fine tuning data

In [2]:
import sys
from BART_utilities import *
sys.path.insert(0, '../')
from transformers import Trainer, TrainingArguments
import pytorch_lightning as pl

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
tldr = pd.read_json('../../../dataset/legal_summarization/tldrlegal_v1.json')

In [10]:
tldr = tldr.transpose().reset_index()

In [11]:
tldr.rename(columns = {'original_text':'source', 'reference_summary':'target'}, inplace = True)

In [12]:
tldr_select = tldr[['source', 'target']]

In [13]:
tldr_select.head()

,source,target
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [14]:
# Add special tokens if required

new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50272. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(50272, 1024)

In [15]:
summary_data = SummaryDataModule(tokenizer, tldr_select, batch_size = 1)
model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [36]:
trainer = pl.Trainer(max_epochs = 3,
                     min_epochs = 2,
                     precision = 16)

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning_fabric\connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:552: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with

In [37]:
trainer.fit(model, summary_data)


Missing logger folder: c:\Users\Asus\Documents\hackathon\trainer bart\lightning_logs
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 406 M 
-------------------------------------------------------
406 M     Trainable params
0         Non-trainable params
406 M     Total params
1,625.194 Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 2: 100%|██████████| 51/51 [21:02<00:00,  0.04it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 51/51 [22:15<00:00,  0.04it/s, v_num=0]


In [38]:
#Save the model
trainer.save_checkpoint("output.ckpt")

In [77]:
trainer.test(model=None, dataloaders=summary_data, ckpt_path='best', verbose=True, datamodule=None)

Restoring states from the checkpoint path at c:\Users\Asus\Documents\hackathon\trainer bart\lightning_logs\version_0\checkpoints\epoch=2-step=153.ckpt
Loaded model weights from the checkpoint at c:\Users\Asus\Documents\hackathon\trainer bart\lightning_logs\version_0\checkpoints\epoch=2-step=153.ckpt
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Validation DataLoader 0: 100%|██████████| 17/17 [02:37<00:00,  0.11it/s]


[{}]

In [16]:
loaded_model = LitModel.load_from_checkpoint("output.ckpt", learning_rate=2e-5, tokenizer=tokenizer, model=bart_model)


In [90]:
loaded_model.eval()

LitModel(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50272, 1024)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50272, 1024)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
           

In [111]:
test_dataloader = summary_data.test_dataloader()


In [129]:
from torch.utils.data import TensorDataset, DataLoader
from rouge import Rouge

# Initialize ROUGE scorer
rouge = Rouge()

# Initialize lists to store generated summaries and reference summaries
generated_summaries = []
reference_summaries = []

# Iterate through the test dataloader
for input_ids, attention_mask, labels in test_dataloader:
    
    print(labels)
    # Move tensors to the appropriate device (if using GPU)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    # Move tensors to the appropriate device (if using GPU)
    input_ids = {'input_ids': input_ids,
                 'attention_mask': attention_mask}

    # Generate summaries for each batch
    # Replace this line with your actual method to generate summaries
    generated_batch = model.generate_text(input_ids, eval_beams=5)

    # Collect the generated and reference summaries
    generated_summaries.extend(generated_batch)
    reference_summaries.extend(labels)  # Assuming labels are the reference summaries

# Compute ROUGE scores
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print ROUGE scores
print("ROUGE Scores:", rouge_scores)


tensor([[    0,   218,   326,  4887, 39121,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

: 

In [17]:
sample_input = "any feedback you provide at this site shall be deemed to be non confidential. apple shall be free to use such information on an unrestricted basis."
inputs = tokenizer(sample_input, return_tensors="pt", max_length=512, truncation=True)
generated_text = loaded_model.generate_text(inputs, eval_beams=5) 

# Print or use the generated summary as needed
print("Generated Summary:", generated_text)

{'input_ids': tensor([[    0,   143,  6456,    47,   694,    23,    42,  1082,  5658,    28,
          7661,     7,    28,   786, 12143,     4, 15162,  5658,    28,   481,
             7,   304,   215,   335,    15,    41, 28225,  1453,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}
Generated Summary: [' apple may use your feedback without restrictions e g. share it publicly.']


In [110]:
from rouge import Rouge

# Compute ROUGE scores
rouge = Rouge()
scores = rouge.get_scores('apple may use your feedback without restrictions e g. share it publicly.', sample_input, avg=True)

# Print ROUGE scores
print("ROUGE Scores:", scores)


ROUGE Scores: {'rouge-1': {'r': 0.13636363636363635, 'p': 0.25, 'f': 0.17647058366782017}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.13636363636363635, 'p': 0.25, 'f': 0.17647058366782017}}
